<a href="https://colab.research.google.com/github/luiz10luiz10/projetoAiBuiderss/blob/main/_1_RAG_AI_Builders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Baixando e instalando as bibliotecas da openai. o --quiet é pra não aparecer o monte de coisas que é instalado e fica oculto
!pip install openai --quiet

In [2]:
# fazendo os imports e puxando a chave api da openai

from google.colab import userdata
userdata.get('openai_api_key')

import openai
import pandas as pd
import numpy as np

from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai_api_key'))


In [4]:
#Fazendo o Embeding exemplo
response = client.embeddings.create(
    input="fazendo teste do embeding",#essa fase de entrada qualquer e que o modelo vai fazer o embeding
    model="text-embedding-3-small"  # esse é um dos modelos da openai verificar outros https://platform.openai.com/docs/guides/embeddings
)

print(response.data[0].embedding)

[0.0063987914472818375, -0.00978448148816824, 0.04398480802774429, -0.013128052465617657, -0.00721848476678133, 0.00997239537537098, 0.0010894144652411342, -0.003900832962244749, -0.017871256917715073, -0.019257932901382446, 0.05049051344394684, 0.024078896269202232, -0.0030811396427452564, -0.015823643654584885, 0.016938168555498123, -0.0065705059096217155, -0.026930002495646477, -0.00543654290959239, 0.005877168383449316, 0.02358643151819706, 0.017275117337703705, -0.020139183849096298, -0.018739549443125725, -0.0024623197969049215, 0.021124111488461494, -0.05520780012011528, -0.0029061853419989347, 0.023832663893699646, 0.009667845442891121, -0.017845338210463524, -0.0025627564173191786, -0.02998846210539341, 0.013374283909797668, 0.0013866748195141554, -0.031543612480163574, 0.000877201440744102, -0.033876337110996246, -0.0025271177291870117, -0.010607413947582245, 0.009564168751239777, 0.0237678661942482, 0.015136786736547947, -0.017275117337703705, 0.021474020555615425, 0.0421186

## Exemplo - Importaremos um arquivo com uma lista de palavras

In [14]:
#Lendo o arquivo cvs que subi

df = pd.read_csv('ingles_teste.csv')


print(df)

          text
0        apple
1   strawberry
2        peach
3       banana
4        mango
5         pear
6       orange
7    pineapple
8         plum
9        grape
10  watermelon
11     apricot
12        kiwi
13      cherry
14       lemon


## Preparando Funções
- Get Embedding para transformar texto em vetores
- Cosine Similarity para buscar por similaridade nos vetores

https://en.wikipedia.org/wiki/Cosine_similarity

In [15]:
#Essa função pega um texto (como uma frase ou parágrafo) e transforma
#ele em um código especial chamado "embedding". Esse "embedding"


def get_embedding(text, model="text-embedding-3-small"): #chamando o modelo aser usado
   text = text.replace("\n", " ")  # aqui ta fazendo a substituiçao da pulada de linha por nada por vazio
   print("Text: ", text)
   response = client.embeddings.create(
       input=text,
       model=model
    )
   return response.data[0].embedding

def cosine_similarity(a, b): # funão criada para calcular a semelhanças dos embeding das palavras.
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [16]:
df["embedding"] = df["text"].apply(lambda x: get_embedding(x))  # criando uma nova coluna com os valores de embeding de cada palavra
df.to_csv("english_text_embeddings.csv") # salvando um novo csv. Esse seria como ficaria um banco de dados vetorial

Text:  apple
Text:  strawberry
Text:  peach
Text:  banana
Text:  mango
Text:  pear
Text:  orange
Text:  pineapple
Text:  plum
Text:  grape
Text:  watermelon
Text:  apricot
Text:  kiwi
Text:  cherry
Text:  lemon


In [23]:
busca = input("Digite uma palavra: ") # utilizando a função | digita a palava que quer fazer o embeding

Digite uma palavra: lemon


In [24]:
busca_vetor = get_embedding(busca) # vai bucar no vetor a palavra mais proxima da que foi digita

Text:  lemon


In [25]:
df["similarity"] = df["embedding"].apply(lambda x: cosine_similarity(x, busca_vetor))# aqui vai ser aplicada a função consine_similarity(x = emebeding da palavra , busca vetor = é o vetor gerado   )
df.sort_values("similarity", ascending=False)# mostrando o a similaridade quanto mais perto de 1 é que tem a ver com a plavra e classificando do mais proximo da palavra

,text,embedding,similarity
14,lemon,"[0.006762837525457144, -0.03722355142235756, -...",1.000000
2,peach,"[0.030673496425151825, -0.04195955768227577, -...",0.478717
10,watermelon,"[0.004471711814403534, -0.011339099146425724, ...",0.473188
1,strawberry,"[0.02010619454085827, -0.014350146055221558, -...",0.470408
13,cherry,"[0.03621857613325119, -0.04493125528097153, -0...",0.451007
6,orange,"[-0.02592204324901104, -0.00554656470194459, -...",0.442187
4,mango,"[0.0554557740688324, -0.008839108981192112, -0...",0.430758
7,pineapple,"[0.02098206616938114, -0.06056341901421547, -0...",0.416961
11,apricot,"[-6.806243618484586e-05, -0.009986917488276958...",0.404468
0,apple,"[0.01764063909649849, -0.016817327588796616, -...",0.390071
